In [4]:
module Queue where

import Prelude hiding (head, tail)

class Queue q where
    empty :: q a
    isEmpty :: q a -> Bool
    snoc :: q a -> a -> q a
    head :: q a -> a
    tail :: q a -> q a

In [12]:
:l Queue.hs

In [13]:
module BatchedQueue where

import Prelude hiding (head, tail)
import Queue

data BatchedQueue a = BQ [a] [a]

check [] r = BQ (reverse r) []
check f r = BQ f r

instance Queue BatchedQueue where
    empty = BQ [] []
    
    isEmpty (BQ f r) = null f
    
    snoc (BQ f r) x = check f (x : r)
    
    head (BQ [] _) = error "Error: Empty Queue"
    head (BQ (x : f) _) = x
    
    tail (BQ [] _) = error "Error: Empty Queue"
    tail (BQ (x : f) r) = check f r

In [15]:
module Dequeue where

import Prelude hiding (head, tail)

class Dequeue q  where
    empty :: q a
    isEmpty :: q a -> Bool
    
    cons :: a -> q a -> q a
    head :: q a -> a
    tail :: q a -> q a
    
    snoc :: q a -> a -> q a
    last :: q a -> a
    init :: q a -> q a

In [16]:
:l Dequeue.hs

In [ ]:
--Exercise 5.1
module BatchedDequeue where

import Prelude hiding (head, tail)
import Dequeue

data BatchedDQ a = DQ [a] [a]

breakHalfRev :: [a] -> ([a], [a])
breakHalfRev lst = (f, r)
    where 
        half = (length lst) `div` 2
        (f, r_rev) = splitAt half lst
        (f, r) = (f, reverse r_rev)

check ::  [a] -> [a] -> BatchedDQ a
check [] r@(x : y : xs) = let (f', r') = breakHalfRev r in DQ f' r'
check f@(x : y : xs) [] = let (f', r') = breakHalfRev f in DQ f' r'
check f r = DQ f r

instance Dequeue BatchedDQ where
    empty = DQ [] []
    isEmpty (DQ f r) = and . map null $ [f, r]
    
    cons x (DQ f r) = DQ (x : f) r
    head (DQ [] []) = error "Error: Empty Dequeue"
    head (DQ [] [y]) = y
    head (DQ f@(x : _) _) = x
    tail (DQ [] []) = error "Error: Empty Dequeue"
    tail (DQ [] [y]) = empty
    tail (DQ f@(x : f') r) = check f' r
    
    snoc (DQ f r) x = DQ f (x : r)
    last (DQ [] []) = error "Error: Empty Dequeue"
    last (DQ [x] []) = x
    last (DQ _ r@(y : _)) = y
    init (DQ [] []) = error "Error: Empty Dequeue"
    init (DQ [x] []) = empty
    init (DQ f r@(y : r')) = check f r'

In [1]:
:l Heap.hs

In [22]:
module SplayHeap where

import Heap

data SplayHeap a = E | T (SplayHeap a) a (SplayHeap a) 

--Exercise 5.4
smaller :: Ord a => a -> SplayHeap a -> SplayHeap a
smaller pivot E = E
smaller pivot (T a x b) =
    if x >= pivot 
    then smaller pivot a
    else
        case b of 
            E -> T a x E
            T b1 y b2 -> if y >= pivot then T a x (smaller pivot b1) else T (smaller pivot b2) y (T a x b1)
            
bigger :: Ord a => a -> SplayHeap a -> SplayHeap a
bigger pivot E = E
bigger pivot (T a x b) = 
    if x <= pivot 
    then bigger pivot b
    else
        case a of 
            E -> T E x b
            T a1 y a2 -> if y <= pivot then T (bigger pivot a2) x b else T (bigger pivot a1) y (T a2 x b)

partition :: Ord a => a -> SplayHeap a -> (SplayHeap a, SplayHeap a)
partition pivot E = (E, E)
partition pivot t@(T a x b) = 
    if x <= pivot then
        case b of 
            E -> (t, E)
            T b1 y b2 -> 
                if y <= pivot then
                    let (small, big) = partition pivot b2
                    in (T (T a x b1) y small, big)
                else
                    let (small, big) = partition pivot b1
                    in (T a x small, T big y b2)
    else
        case a of
            E -> (E, t)
            T a1 y a2 -> 
                if y <= pivot then
                    let (small, big) = partition pivot a2
                    in (T a1 y small, T big x b)
                else
                    let (small, big) = partition pivot a1
                    in (small, T big y (T a2 x b))

instance Heap SplayHeap where
    empty = E
    
    isEmpty E = True
    isEmpty _ = False

    insert x t = T small x big
        where (small, big) = partition x t

    merge E t = t
    merge (T a x b) t = T (merge ta a) x (merge tb b)
        where (ta, tb) = partition x t

    findMin (T E x b) = x
    findMin (T a x b) = findMin a

    deleteMin (T E x b) = b
    deleteMin (T (T E x b) y c) = T b y c
    deleteMin (T (T a x b) y c) = T (deleteMin a) x (T b y c)

In [23]:
:l SplayHeap.hs

In [24]:
--Exercise 5.7
import SplayHeap

sortWSplayHeap :: Ord a => [a] -> [a]
sortWSplayHeap = inorderTraverse . foldr insert E

inorderTraverse :: Ord a => SplayHeap a -> [a]
inorderTraverse E = []
inorderTraverse t = findMin t : (inorderTraverse . deleteMin $ t)

main :: IO ()
main = do
    print $ sortWSplayHeap [1,2,5,4,3,6,8,7]
    
main

Line 12: Redundant do
Found:
do print $ sortWSplayHeap [1, 2, 5, 4, 3, 6, 8, 7]
Why not:
print $ sortWSplayHeap [1, 2, 5, 4, 3, 6, 8, 7]

: 

In [1]:
:l Heap.hs

In [2]:
module PairingHeap where

import Heap

data PairingHeap a = E | T a [PairingHeap a] deriving (Show)

mergePairs :: Ord a => [PairingHeap a]  -> PairingHeap a
mergePairs [] = E
mergePairs [h] = h
mergePairs (h1 : h2 : hs) = merge (merge h1 h2) (mergePairs hs)

instance Heap PairingHeap where
    empty = E
    
    isEmpty E = True
    isEmpty _ = False
    
    merge h E = h
    merge E h = E
    merge h1@(T x hs1) h2@(T y hs2) = 
        if x <= y 
        then T x (h2 : hs1)
        else T y (h1 : hs2)
        
    insert x h = merge (T x []) h
    
    findMin E = error "Error: Empty Heap"
    findMin (T x hs) = x
    
    deleteMin E = error "Error: Empty Heap"
    deleteMin (T x hs) = mergePairs hs

Line 25: Eta reduce
Found:
insert x h = merge (T x []) h
Why not:
insert x = merge (T x [])

In [14]:
--Exercise 5.8
:l PairingHeap.hs

import Heap
import qualified PairingHeap as P

data BinTree a = E | T a (BinTree a) (BinTree a) deriving (Show)

toBinary :: P.PairingHeap a -> BinTree a
toBinary P.E = E
toBinary (P.T x hs) = T x (toBinaryNode hs) E

toBinaryNode :: [P.PairingHeap a] -> BinTree a
toBinaryNode [] = E
toBinaryNode (P.T x hsx : hs) = T x (toBinaryNode hsx) (toBinaryNode hs)

mergePairs :: Ord a => BinTree a -> BinTree a
mergePairs E = E
mergePairs (T x lt E) = T x lt E
mergePairs (T x lt (T y lt' rt')) = merge (merge (T x lt E) (T y lt' E)) (mergePairs rt')

instance Heap BinTree where
    empty = E
    
    isEmpty E = True
    isEmpty _ = False
    
    merge t E = t
    merge E t = t
    merge t1@(T x1 lt1 E) t2@(T x2 lt2 E) =
        if x1 <= x2 
        then T x1 (T x2 lt2 lt1) E
        else T x2 (T x1 lt1 lt2) E
        
    insert x t = merge (T x E E) t
    
    findMin E = error "Error: Empty Heap"
    findMin (T x _ _) = x
    
    deleteMin E = error "Error: Empty Heap"
    deleteMin (T x lt E) = mergePairs lt 